# Linear Regression

Why use linear regression?

1. Easy to use
2. Easy to interpret
3. Basis for many methods
4. Runs fast
5. Most people have heard about it :-) 

### Libraries in Python for Linear Regression

The two most popular ones are

1. `scikit-learn`
2. `statsmodels`

We highly recommend learning `scikit-learn` since that's also the machine learning package in Python.

## Problem

Could we predict price of weed in a state using the demographic information? 

For this session, let's do this. For January 2015, let's find average price for high quality weed across all the states. Let's assume that we don't know what the prices are for the following states: 
<br>
iowa, kentucky, missouri, nevada, wyoming, south dakota, new jersey, michigan, idaho
<br>

Those are our **test** set. The remaining states are our **train** set. We need to train the model on the train dataset and predict for the test dataset. 

Since we also know the actual mean prices for the test states, let's verify how good our models are.

In [ ]:
import pandas as pd

In [ ]:
#Load the data
weed_pd = pd.read_csv("../data/Weed_Price.csv", parse_dates=[-1])
demo_pd = pd.read_csv("../data/Demographics_State.csv")

In [ ]:
weed_pd.head()

In [ ]:
weed_pd.dtypes

In [ ]:
demo_pd.head()

In [ ]:
demo_pd.dtypes

As seen above, weed price dataset has states' first alphabet capitalized. 
The below command will convert it to lower case

In [ ]:
str.lower("Alabama")

In [ ]:
?weed_pd.apply

In [ ]:
weed_pd.State = weed_pd["State"].apply(lambda x: str.lower(x))

In [ ]:
weed_pd.head()

In [ ]:
pd.unique(weed_pd.State)

In [ ]:
pd.unique(demo_pd.region)

In [ ]:
#Let's get month and year of the date, so that we can select only Jan 2015 data
weed_pd["Month"] = weed_pd["date"].apply(lambda x: x.month)
weed_pd["Year"] = weed_pd["date"].apply(lambda x: x.year)

In [ ]:
weed_jan2015_pd = weed_pd.ix[(weed_pd.Year==2015) & (weed_pd.Month==1)]

In [ ]:
weed_jan2015_pd.head()

In [ ]:
weed_jan2015_summarized = weed_jan2015_pd[["State", "HighQ"]].groupby("State").mean().reset_index()

In [ ]:
#The source price dataset for our model 
weed_jan2015_summarized

In [ ]:
test_states = ["iowa", "kentucky", "missouri", "nevada", "wyoming", \
               "south dakota", "new jersey", "michigan", "idaho" ]

In [ ]:
data_for_model = pd.merge(weed_jan2015_summarized, demo_pd, left_on="State", right_on="region")

In [ ]:
data_for_model.head()

In [ ]:
#Now, creating train and test dataset
criterion = weed_jan2015_summarized["State"].map(lambda x: x in test_states)
#Another way to do it
#criterion = weed_jan2015_summarized.State.isin(test_states)

In [ ]:
print "Train data labels: \n", ~criterion, "\n\n", "Test data labels: \n", criterion

In [ ]:
train = data_for_model[~criterion]
test = data_for_model[criterion]

In [ ]:
train.shape, test.shape

In [ ]:
train

In [ ]:
test

### Linear regression 

Let's use `statsmodels` for this workshop. 

Linear regression is of the form:

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$

- $y$ is what we have the predict/independent variable/response variable
- $\beta_0$ is the intercept/slope
- $\beta_1$ is the coefficient for $x_1$ (the first feature/dependent variable)
- $\beta_n$ is the coefficient for $x_n$ (the nth feature/dependent variable)

The $\beta$ are called *model coefficients*

The model coefficients are estimated in this process. (In Machine Learning parlance - the weights are learned using the algorithm). The objective function is least squares method. 
<br>

**Least Squares Method** : To identify the weights so that the overall solution minimizes the sum of the squares of the errors made in the results of every single equation. [Wiki](https://en.wikipedia.org/wiki/Least_squares)

![Estimating coefficients](img/leastsquare.gif)


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

In [ ]:
#First step: Let's visualize bivariate plots with  a trend line

In [ ]:
sns.regplot(x="percent_white", y = "HighQ", data=train)

In [ ]:
sns.regplot(x="total_population", y="HighQ", data=train)

**Exercise** Plot for `HighQ` vs `per_capita_income` 

**Fitting another kind of plot/model**

In [ ]:
?sns.lmplot

In [ ]:
sns.lmplot(x="total_population", y="HighQ", data=train, order=3)

In [ ]:
#More plots.

#Visualizing correlation matrix using a heatmap

sns.heatmap(train.corr())

In [ ]:
?sns.pairplot

In [ ]:
#Multiple scatter plot

sns.pairplot(train, x_vars='HighQ', y_vars=['total_population', 'percent_white', 'percent_black', \
                                           'per_capita_income', 'median_rent', 'median_age'], kind='reg')

**First, let's build a single variable model.**

Let's try to estimate price as a function of population

In [ ]:
feature_columns = ["total_population"]
train_x = train[feature_columns]
train_y = train['HighQ']

In [ ]:
model_1 = LinearRegression()
model_1.fit(train_x, train_y)

In [ ]:
print model_1.intercept_
print model_1.coef_

**Let's use the model for prediction**

In [ ]:
model_1_predict = model_1.predict(test[feature_columns])

In [ ]:
model_1_predict = pd.DataFrame({'States': test.State, 'Actual Price': test.HighQ, 'Predicted Price': model_1_predict})

In [ ]:
model_1_predict

**Computing mean squared error**

In [ ]:
#Root Mean square error on test dataset
np.sqrt(np.mean(np.square(model_1_predict['Actual Price'] - \
                         model_1_predict['Predicted Price'])))

**Lower the RMSE, better the model **

**Exercise: Create model using `total_population` and `per_capita_income` as the features. Report RMSE**

### Pointers

To build the model the right way, the following need to be done. They are left as exercise. 

1. Scale the features
2. Use cross-validation
3. As more features are added, the model becomes complicated and would overfit the data. Will need regularization
4. Try feature transformation to see if lower RMSE is possible. 

### Why wasn't **R-square** and **p-value** covered? 

- Linear models rely upon a lot of assumptions ([here](http://andrewgelman.com/2013/08/04/19470/)). If assumptions are violated, the diagnostics obtained from the model cannot be relied. 
- Biggest challenge is that adding any feature will increase the R-square. One way to counter this is to use adjusted R-squre.
- Take a step back and think - why do we need to report those numbers? We want some estimate of generalization. Cross-validation score provides a general framework for reporting generalization. And this will hold good across all models. And thus, multiple models can be compared. This is the machine learning approach and is widely used in practice.